# RNN Play Generator
Create a play using RNN
- Show the RNN an example of what we want it to recreate and it will learn how to write a version of it on its own.

In [2]:
from keras.preprocessing import sequence
import tensorflow as tf 
import numpy as np 
import os

We are going to train the model on a bunch of sequences of texts from the play "Romeo and Juliet" 

In [3]:
path_to_file = tf.keras.utils.get_file("shakespeare.txt","https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt")

In [4]:
# Read and then decode for py2 compat
text = open(path_to_file, "rb").read().decode(encoding="utf-8")
print("Length of text: {} characters".format(len(text))) # result same as downloaded as you can see over.

Length of text: 1115394 characters


In [5]:
# The first 250 characters in the text
print(text[:250]) 

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



Encode each unique character as a different integer.

In [6]:
vocab = sorted(set(text))
# Create a mapping from unique characters to indices
char2idx = {u:i for i,u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
    return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [7]:
print("Text: ", text[:13])
print("Encoded: ", text_to_int(text[:13]))

Text:  First Citizen
Encoded:  [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [8]:
def int_to_text(ints):
    try:
        ints = ints.numpy()
    except:
        pass
    return "".join(idx2char[ints])
print(int_to_text(text_as_int[:13]))

First Citizen


### Creating a training example
We need to split our text data from above into many shorter sequences that we can pass to the model as training example.

The training example will use a *seq_length* sequence as input and a *seq_length* sequence as the output where the sequence is the original sequence shifted one letter to the right. Example:
- Input: Hell
- Output: ello
We could then splice it on the last index and then see that it predicts "o" as the last char for the actual word, "Hello".

In [9]:
seq_length = 100 # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

Now we can use the batch method to turn this stream of characters into batches of desired length

In [10]:
sequences = char_dataset.batch(seq_length+1,drop_remainder=True)

In [11]:
def split_input_target(chunk): # for the example: hello
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [12]:
for x,y in dataset.take(2):
    print("\n\nEXAMPLE\n")
    print("INPUT")
    print(int_to_text(x))
    print("\nOUTPUT")
    print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [13]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab) # number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [14]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer="glorot_uniform"),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5330241 (20.33 MB)
Trainable params: 5330241 (20.33 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Creating a loss function

In [15]:
for input_example_batch, target_example_batch in data.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [16]:
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 1.13420701e-03  9.08431772e-04 -2.94142449e-03 ... -2.29606964e-03
    2.45451462e-03 -2.10238784e-03]
  [ 1.20507274e-02 -9.85535327e-04 -2.11550016e-03 ... -2.41221613e-04
   -5.53662842e-03 -1.95679744e-03]
  [ 1.03269797e-02  2.05006078e-03 -5.27716102e-03 ...  3.19400779e-03
    5.92108467e-04 -1.49707869e-03]
  ...
  [ 4.78191348e-03  5.79812890e-03  5.60208131e-03 ... -1.06012020e-02
    5.12998784e-03 -4.86677838e-03]
  [ 3.94266006e-03  6.98575098e-03  7.62356911e-04 ... -6.14348520e-03
    9.28916316e-03 -2.48014834e-03]
  [ 5.24542946e-03  5.92864258e-03 -3.19685671e-04 ... -4.92570456e-03
    1.08414590e-02 -2.57315871e-04]]

 [[ 5.22475736e-03  9.99243231e-04 -1.61054346e-03 ...  5.14486246e-03
    1.67520822e-03 -7.39743561e-03]
  [ 5.20211877e-03  7.48357095e-04  4.01893398e-03 ...  1.52268517e-03
    8.72902689e-04 -1.09827518e-02]
  [ 4.74802777e-03  2.79178284e-03  2.05213437e-05 ...  3.50705162e-03
    6.43235818e-03 -6.83996826e-03]
  ...
  [-2.738

In [17]:
# Examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)

100
tf.Tensor(
[[ 0.00113421  0.00090843 -0.00294142 ... -0.00229607  0.00245451
  -0.00210239]
 [ 0.01205073 -0.00098554 -0.0021155  ... -0.00024122 -0.00553663
  -0.0019568 ]
 [ 0.01032698  0.00205006 -0.00527716 ...  0.00319401  0.00059211
  -0.00149708]
 ...
 [ 0.00478191  0.00579813  0.00560208 ... -0.0106012   0.00512999
  -0.00486678]
 [ 0.00394266  0.00698575  0.00076236 ... -0.00614349  0.00928916
  -0.00248015]
 [ 0.00524543  0.00592864 -0.00031969 ... -0.0049257   0.01084146
  -0.00025732]], shape=(100, 65), dtype=float32)


In [18]:
# a prediction for the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)

65
tf.Tensor(
[ 1.1342070e-03  9.0843177e-04 -2.9414245e-03 -3.0524228e-03
 -3.5438510e-03 -3.0541301e-03 -2.9473910e-03 -1.4797676e-03
  3.8835178e-03  4.1887634e-03 -1.9365759e-04  3.7354336e-03
 -1.4550934e-03 -1.0582129e-03  3.5190010e-03  4.1864361e-03
 -2.5304079e-03 -2.0925761e-03 -3.5008551e-03  2.9958473e-03
  1.6193232e-03 -1.7115383e-03  1.3750441e-03 -3.8149231e-03
 -2.1390663e-03 -1.4467712e-03 -3.6224970e-03  8.1986813e-03
 -2.6555749e-04 -2.8163185e-03  1.3159646e-05  9.2630193e-04
  1.6621489e-03  4.2922446e-05  3.1706186e-03  6.4774007e-03
  2.4956942e-03 -1.5643283e-03 -6.8764407e-03 -3.0439172e-03
  2.0888292e-03  4.3299990e-03  1.3253761e-03  4.9753534e-03
  4.5648045e-03  5.4711271e-03  3.0770786e-03  3.0528968e-03
  2.1827032e-03 -7.9867970e-03 -1.7298812e-03  4.5429971e-03
  5.5826786e-03  3.3996128e-03 -3.0020189e-03  2.4906220e-03
  5.4657166e-03 -5.5148907e-04 -1.2552382e-04  5.7934155e-03
  3.8018945e-04  3.2390123e-03 -2.2960696e-03  2.4545146e-03
 -2.102387

This shows the probability of the *next* character for the first timestep

In [19]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probability)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# then reshape the array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices,(1,-1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars

".W$t!mHWmeRjtBAIjynHMRw\n3YZ?Ty?S'HJ'tvQAVx.B.B,bUvsJs!cdFR'VixmBRHWwlEwrhiubu$CIcvqDJhjAXfnn:cQJuLyi"

In [20]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [21]:
model.compile(optimizer="adam", loss=loss)

### Creating checkpoints

In [22]:
# Directory where the checkpoints will be saved
checkpoint_dir = "./training_checkpoints"
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only = True
)

In [23]:
# This took 15 mins, with more epochs the model becomes much better!
history = model.fit(data, epochs=5, callbacks=[checkpoint_callback])

Epoch 1/5


172/172 [==============================] - 194s 1s/step - loss: 2.5960
Epoch 2/5
172/172 [==============================] - 192s 1s/step - loss: 1.8877
Epoch 3/5
172/172 [==============================] - 188s 1s/step - loss: 1.6372
Epoch 4/5
172/172 [==============================] - 186s 1s/step - loss: 1.5041
Epoch 5/5
172/172 [==============================] - 185s 1s/step - loss: 1.4242


### Loading the model
Rebuild the model from a checkpoint using a batch_size of 1 so that we can feed one piece of text to the model and have it make a prediction.

In [24]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

Load any checkpoint by specifying the exact file to load

In [29]:
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
model.load_weights(latest_checkpoint)
model.build(tf.TensorShape([1, None]))

In [39]:
def generate_text(model, start_string):
    # Number of characters to generate
    num_generate = 800

    # Convert start string to numbers (vectorizing)
   # Convert start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    # Here begins the temperature setup and the reset of the model state
    temp = 1.0
    model.reset_states()

    for _ in range(num_generate):
        # This is where predictions are made
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        # Here the predictions are divided by the temperature and a random choice is made
        predictions = predictions / temp
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Prepare the predicted_id for the next round of prediction
        input_eval = tf.expand_dims([predicted_id], 0)

        # And here the predicted character is added to the generated text
        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))


In [40]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Victoria! now thus you not peagled fall,
Ternater from eleaves wak, my father's souly!

QUEEN ELIZABETH:
A Rabenhal, at these fore Cariall'd thoughtship's sensw,
For I moytal sleep pritaye under: day,
That are both your dopsor traits: stay it
poss'station, hot do boild thy almoty worse:
Here's my old hoaring that Ramiot of yourself.
then, loth amesitoon Pet up,
My man tolpy hesold no whise,
But when these norm is basely by no
man? How thou hasits, away not spuding an your hemp,
For vain cannot again; you wert from the king,
That you brother like perdfulms stands, pritter, Demember,
It scape against their squeless of these you speding enemy.

POLIXENES:
You pleased thempell:
Then strengthing upon your galous, of cousins may
Therefore me a woo, gry sovereign:
Let's to so best I go to off,
Comstakene
